# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [3]:
import findspark
findspark.init('/home/gerardo-rodriguez/spark-4.0.0-bin-hadoop3')

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lr_ship').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/18 13:12:09 WARN Utils: Your hostname, Lanz-Lenovo, resolves to a loopback address: 127.0.1.1; using 192.168.1.145 instead (on interface wlp2s0)
25/08/18 13:12:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/18 13:12:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/18 13:12:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/18 13:12:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/08/18 13:12:11 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [18]:
from pyspark.ml.regression import LinearRegression

In [15]:
data = spark.read.csv('cruise_ship_info.csv', header=True, inferSchema=True)

In [16]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [20]:
from pyspark.ml.linalg import Vector as v
from pyspark.ml.feature import VectorAssembler as va
from pyspark.ml.feature import StringIndexer as si

In [23]:
indexer = si(inputCol='Cruise_line', outputCol='cruise_line')
data_indexed = indexer.fit(data).transform(data)

In [24]:
data_indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|       16.0|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|       16.0|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|        1.0| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|        1.0| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|        1.0| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|        1.0| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|        1.0| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|        1.0| 23| 

In [25]:
data_indexed.columns

['Ship_name',
 'cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [26]:
assembler = va(inputCols=['cruise_line',
                         'Age',
                         'Tonnage',
                         'passengers',
                         'length',
                         'cabins',
                         'passenger_density',
                         'crew'], 
                outputCol='features')

In [28]:
output = assembler.transform(data_indexed)

In [29]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- cruise_line: double (nullable = false)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- features: vector (nullable = true)



In [30]:
for item in output.head(1)[0]:
    print(item)

Journey
16.0
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55
[16.0,6.0,30.276999999999997,6.94,5.94,3.55,42.64,3.55]


In [31]:
final_data = output.select(['features', 'crew'])

In [32]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[16.0,6.0,30.2769...|3.55|
|[16.0,6.0,30.2769...|3.55|
|[1.0,26.0,47.262,...| 6.7|
|[1.0,11.0,110.0,2...|19.1|
|[1.0,17.0,101.353...|10.0|
|[1.0,22.0,70.367,...| 9.2|
|[1.0,15.0,70.367,...| 9.2|
|[1.0,23.0,70.367,...| 9.2|
|[1.0,19.0,70.367,...| 9.2|
|[1.0,6.0,110.2389...|11.5|
|[1.0,10.0,110.0,2...|11.6|
|[1.0,28.0,46.052,...| 6.6|
|[1.0,18.0,70.367,...| 9.2|
|[1.0,17.0,70.367,...| 9.2|
|[1.0,11.0,86.0,21...| 9.3|
|[1.0,8.0,110.0,29...|11.6|
|[1.0,9.0,88.5,21....|10.3|
|[1.0,15.0,70.367,...| 9.2|
|[1.0,12.0,88.5,21...| 9.3|
|[1.0,20.0,70.367,...| 9.2|
+--------------------+----+
only showing top 20 rows


In [33]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [35]:
lr = LinearRegression(labelCol='crew')

In [36]:
lr_model = lr.fit(train_data)

25/08/18 13:26:36 WARN Instrumentation: [dbf7e0a8] regParam is zero, which might cause numerical instability and overfitting.


In [37]:
test_result = lr_model.evaluate(test_data)

In [39]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|1.065814103640150...|
|-7.10542735760100...|
|1.776356839400250...|
|2.131628207280300...|
|6.217248937900877...|
|2.398081733190338...|
|2.664535259100375...|
|7.105427357601002...|
|3.730349362740526...|
|1.776356839400250...|
|3.552713678800501...|
|-5.32907051820075...|
|-1.42108547152020...|
|-1.95399252334027...|
|-2.48689957516035...|
|-2.48689957516035...|
|1.598721155460225...|
|-3.19744231092045...|
|-1.42108547152020...|
|-3.37507799486047...|
+--------------------+
only showing top 20 rows


In [40]:
test_result.rootMeanSquaredError

3.7657400412003044e-14

In [41]:
test_result.r2

1.0

In [42]:
unlabeled_data = test_data.select('features')

In [44]:
prediction = lr_model.transform(unlabeled_data)

In [46]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[0.0,4.0,220.0,54...| 20.99999999999999|
|[0.0,7.0,158.0,43...|13.600000000000007|
|[0.0,10.0,90.09,2...| 8.579999999999998|
|[0.0,11.0,138.0,3...|11.849999999999978|
|[0.0,17.0,70.0,20...| 7.199999999999994|
|[0.0,17.0,74.137,...| 7.599999999999976|
|[0.0,22.0,73.941,...| 8.219999999999974|
|[0.0,25.0,73.192,...| 8.079999999999993|
|[1.0,9.0,110.0,29...|11.599999999999962|
|[1.0,11.0,86.0,21...| 9.299999999999999|
|[1.0,13.0,101.509...|11.499999999999996|
|[1.0,15.0,70.367,...| 9.200000000000005|
|[1.0,18.0,70.367,...| 9.200000000000014|
|[1.0,20.0,70.367,...| 9.200000000000019|
|[1.0,22.0,70.367,...| 9.200000000000024|
|[1.0,23.0,70.367,...| 9.200000000000024|
|[2.0,6.0,113.0,37...|11.999999999999984|
|[2.0,9.0,113.0,26...|12.380000000000033|
|[2.0,11.0,91.6270...| 9.000000000000014|
|[2.0,11.0,108.977...|12.000000000000034|
+--------------------+------------